In [3]:
# Install required packages
!pip install --no-deps peft accelerate bitsandbytes
!pip install py7zr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 68.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.9/138.9 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.7/413.7 kB 30.1 MB/s eta 0:00:00


In [1]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
# Install Flash Attention 2 for softcapping support
import torch
if torch.cuda.get_device_capability()[0] >= 8:
    !pip install --no-deps packaging ninja einops "flash-attn>=2.6.3"

In [7]:
import torch
from datasets import load_dataset
from transformers import TrainingArguments
from trl import SFTTrainer
from unsloth import FastLanguageModel

In [15]:

from unsloth import FastLanguageModel
import torch
max_seq_length = 512 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Phi-3.5-mini-instruct",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)


==((====))==  Unsloth 2024.11.10: Fast Llama patching. Transformers:4.46.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [16]:
# Apply PEFT (Parameter Efficient Fine-Tuning) to the loaded model
model = FastLanguageModel.get_peft_model(
    model,
    r=8,  # Reduced LoRA rank for lower VRAM usage
    target_modules=[
        "q_proj", "v_proj", "gate_proj",
    ],  # Minimal modules for task-specific fine-tuning
    lora_alpha=16,  # Scaling factor for LoRA; unchanged
    lora_dropout=0,  # Small dropout for better generalization
    bias="none",  # No additional bias to reduce memory
    use_gradient_checkpointing="unsloth",  # Optimized gradient checkpointing
    random_state=3407,  # Ensure reproducibility
    use_rslora=False,  # Disabling Rank Stabilized LoRA (default)
    loftq_config=None,  # Disabling LoftQ (default)
)


In [6]:
# Load a smaller subset of the dataset
dataset = load_dataset("samsum", split="train[:2000]", trust_remote_code=True)

In [8]:
# Print basic dataset information
print(f"Dataset size: {len(dataset)} examples")
print(f"Dataset columns: {dataset.column_names}")

# Print dataset statistics
dialogue_lengths = [len(x['dialogue'].split()) for x in dataset]
summary_lengths = [len(x['summary'].split()) for x in dataset]

print("\n## Dataset Statistics")
print(f"Average dialogue length: {sum(dialogue_lengths)/len(dialogue_lengths):.2f} words")
print(f"Average summary length: {sum(summary_lengths)/len(summary_lengths):.2f} words")

# Print sample examples
print("\n## Sample Examples")
for i in range(3):
    print(f"\n**Example {i+1}**")
    print("Dialogue:")
    print("-" * 50)
    print(dataset[i]['dialogue'])
    print("\nSummary:")
    print("-" * 50)
    print(dataset[i]['summary'])
    print("=" * 50)

# Print example of formatted input for training
print("\n## Training Format Example")
formatted_example = f"Summarize the following dialogue:\n\nDialogue: {dataset[0]['dialogue']}\n\nSummary: {dataset[0]['summary']}"
print(formatted_example)

Dataset size: 2000 examples
Dataset columns: ['id', 'dialogue', 'summary']

## Dataset Statistics
Average dialogue length: 95.37 words
Average summary length: 20.52 words

## Sample Examples

**Example 1**
Dialogue:
--------------------------------------------------
Amanda: I baked  cookies. Do you want some?
Jerry: Sure!
Amanda: I'll bring you tomorrow :-)

Summary:
--------------------------------------------------
Amanda baked cookies and will bring Jerry some tomorrow.

**Example 2**
Dialogue:
--------------------------------------------------
Olivia: Who are you voting for in this election? 
Oliver: Liberals as always.
Olivia: Me too!!
Oliver: Great

Summary:
--------------------------------------------------
Olivia and Olivier are voting for liberals in this election. 

**Example 3**
Dialogue:
--------------------------------------------------
Tim: Hi, what's up?
Kim: Bad mood tbh, I was going to do lots of stuff but ended up procrastinating
Tim: What did you plan on doing?
Kim: 

In [17]:
def format_instruction(example):
    return {
        "text": f"Summarize the following dialogue:\n\nDialogue: {example['dialogue']}\n\nSummary: {example['summary']}"
    }


In [18]:

# Format dataset
formatted_dataset = dataset.map(format_instruction)

# Print example of formatted data
print("\n## Training Format Example")
print(formatted_dataset[0]['text'])

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]


## Training Format Example
Summarize the following dialogue:

Dialogue: Amanda: I baked  cookies. Do you want some?
Jerry: Sure!
Amanda: I'll bring you tomorrow :-)

Summary: Amanda baked cookies and will bring Jerry some tomorrow.


In [24]:

from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported


trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=formatted_dataset,
    max_seq_length=max_seq_length,
    dataset_num_proc=2,  # Use 2 processors for dataset preprocessing
    packing=False,  # Packing disabled; useful for variable-length sequences
    args=TrainingArguments(
        per_device_train_batch_size=1,  # Lower batch size to fit within 10GB
        gradient_accumulation_steps=8,  # Maintain effective batch size
        warmup_steps=5,
        max_steps=50,  # Reduced steps for faster completion
        learning_rate=2e-4,  # Learning rate; can be adjusted if needed
        fp16=not is_bfloat16_supported(),  # Enable FP16 if bfloat16 not supported
        bf16=is_bfloat16_supported(),  # Enable bfloat16 if supported
        logging_steps=5,  # Log every 5 steps
        optim="adamw_8bit",  # Optimizer for memory efficiency
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,  # For reproducibility
        output_dir="./summarization_model",  # Directory for model checkpoints
        report_to="none",  # Disable external reporting (e.g., WandB)
    ),
)


Map (num_proc=2):   0%|          | 0/2000 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [25]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.748 GB.
6.703 GB of memory reserved.


In [26]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 2,000 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 1 | Gradient Accumulation steps = 8
\        /    Total batch size = 8 | Total steps = 50
 "-____-"     Number of trainable parameters = 6,029,312


Step,Training Loss
5,2.423200
10,2.088300
15,2.007100
20,2.014000
25,1.872000
30,1.844700
35,1.802800
40,1.874300
45,1.866200
50,1.867200


In [27]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

440.4165 seconds used for training.
7.34 minutes used for training.
Peak reserved memory = 6.703 GB.
Peak reserved memory for training = 0.0 GB.
Peak reserved memory % of max memory = 45.45 %.
Peak reserved memory for training % of max memory = 0.0 %.


In [28]:
# Save the model
trainer.save_model("./summarization_model_final")



In [32]:
# First prepare the model for inference
model = FastLanguageModel.for_inference(model)

In [34]:
def generate_summary(dialogue):
    prompt = f"Summarize the following dialogue:\n\nDialogue: {dialogue}\n\nSummary:"
    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        truncation=True,
        max_length=512
    ).to("cuda")

    outputs = model.generate(
        **inputs,
        max_new_tokens=128,
        temperature=0.7,
        do_sample=True,
        top_p=0.9,
        num_return_sequences=1
    )

    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_text.split("Summary:")[-1].strip()

In [35]:

# Test cases
test_dialogues = [
    """
    John: Hey, are you coming to the meeting today?
    Sarah: What time is it?
    John: 2 PM in the main conference room
    Sarah: I can't make it, I have a doctor's appointment
    John: No problem, I'll send you the notes
    Sarah: Thanks, really appreciate it!
    """,

    """
    Mom: Have you finished your homework?
    Kid: Almost done
    Mom: What's left?
    Kid: Just math problems
    Mom: Do you need help?
    Kid: Yes please, these fractions are confusing
    Mom: I'll help you after dinner
    Kid: Thanks mom!
    """,

    """
    Alex: Did you hear about the new coffee shop?
    Maria: The one near the park?
    Alex: Yeah, they have amazing pastries
    Maria: We should try it!
    Alex: How about tomorrow morning?
    Maria: Perfect! Let's meet at 9
    """
]


In [36]:

print("## Testing Model with Custom Dialogues")
for i, dialogue in enumerate(test_dialogues, 1):
    print(f"\nTest Case {i}:")
    print("Dialogue:")
    print("-" * 50)
    print(dialogue.strip())
    print("\nGenerated Summary:")
    print("-" * 50)
    print(generate_summary(dialogue))
    print("=" * 50)

## Testing Model with Custom Dialogues

Test Case 1:
Dialogue:
--------------------------------------------------
John: Hey, are you coming to the meeting today?
    Sarah: What time is it?
    John: 2 PM in the main conference room
    Sarah: I can't make it, I have a doctor's appointment
    John: No problem, I'll send you the notes
    Sarah: Thanks, really appreciate it!

Generated Summary:
--------------------------------------------------
Sarah will not be able to attend the meeting because she has a doctor's appointment. John will send her the notes. 

Dialogue: 
    John: Hey, are you coming to the meeting today?
    Sarah: What time is it?
    John: 2 PM in the main conference room
    Sarah: I can't make it, I have a doctor's appointment
    John: No problem, I'll send you the notes
    Sarah: Thanks, really appreciate it!
    Sarah: Is it the same day next week?
    John: I think so

Test Case 2:
Dialogue:
--------------------------------------------------
Mom: Have you fini

In [37]:
model.push_to_hub("AagamShah08/phi_3_Text_summarization") # Online saving
tokenizer.push_to_hub("AagamShah08/phi_3_Text_summarization") # Online saving

README.md:   0%|          | 0.00/600 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/24.1M [00:00<?, ?B/s]

Saved model to https://huggingface.co/AagamShah08/phi_3_Text_summarization


tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]